In [1]:
%load_ext autoreload

In [7]:
import pandas as pd

import os
os.chdir('../scripts')

from data_processing.poquad import *

from t5.load_t5 import load_plt5

In [3]:
train, valid =  load_poquad_datasets("../data/poquad-original")

In [4]:
train_df = train.to_pandas()

In [5]:
train_df = train_df.set_index("id")

In [6]:
valid

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5764
})

In [7]:
valid.to_pandas()["answers"].iloc[0]

{'text': array(['kompilację poglądów różnych rabinów na określony temat'],
       dtype=object),
 'answer_start': array([43], dtype=int32)}

In [8]:
for val in train_df.iloc[0]:
    print(val)

Konfederacja polsko-czechosłowacka
Projekty konfederacji zaczęły się załamywać 5 sierpnia 1942. Ponownie wróciła kwestia monachijska, co uaktywniło się wymianą listów Ripka – Stroński. Natomiast 17 sierpnia 1942 doszło do spotkania E. Beneša i J. Masaryka z jednej a Wł. Sikorskiego i E. Raczyńskiego z drugiej strony. Polscy dyplomaci zaproponowali podpisanie układu konfederacyjnego. W następnym miesiącu, tj. 24 września, strona polska przesłała na ręce J. Masaryka projekt deklaracji o przyszłej konfederacji obu państw. Strona czechosłowacka projekt przyjęła, lecz już w listopadzie 1942 E. Beneš podważył ideę konfederacji. W zamian zaproponowano zawarcie układu sojuszniczego z Polską na 20 lat (formalnie nastąpiło to 20 listopada 1942).
Co było powodem powrócenia konceptu porozumieniu monachijskiego?
{'text': array(['wymianą listów Ripka – Stroński'], dtype=object), 'answer_start': array([117], dtype=int32)}


In [9]:
def generate_answers(df, model_path, colab=False):
    # Load PLT5 model
    tokenizer, model = load_plt5(model_path, colab)

    # Prepare input text
    input_texts = dataset_into_str_input(df).tolist()

    # Tokenize input texts
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)

    # Generate model output
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=50,  # Adjust max_length as needed
        num_beams=4,    # Adjust num_beams as needed
        early_stopping=True
    )

    # Decode model output
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Add generated answers to dataframe
    df["generated_answer"] = decoded_outputs

    return df

In [10]:
result = generate_answers(train_df.iloc[:10], "../models/plt5-original-base")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_14840/2084891990.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["generated_answer"] = decoded_outputs


In [27]:
train_df["answers"].apply(lambda x: x.keys()).value_counts()

answers
(text, answer_start)    46187
Name: count, dtype: int64

In [28]:
valid_df = valid.to_pandas()

In [32]:
valid_df.iloc[4].context

'Sformowany przez nią oddział partyzancki liczył 280 strzelców, kilkuset chłopów kosynierów i 60 kawalerzystów. Wraz z oddziałem rozpoczęła marsz w kierunku Dyneburga, po drodze do oddziału dołączali wciąż ochotnicy. 29 marca 1831 wkroczyła do Dusiatów, gdzie pod rozwiniętą flagą Polski zachęcała okoliczną ludność do przyłączenia się do powstania. 30 marca 1831 wraz ze swym hufcem zajęła stację Daugiele. 2 kwietnia jej oddział stoczył zwycięską potyczkę i zniósł kompanię piechoty rosyjskiej pod Ucianami. 4 kwietnia uderzyła na jedną z kolumn korpusu gen. Schirmana, która zmierzała w rejon koncentracji głównych sił rosyjskich w Dyneburgu. Oddział Emilii Plater opanował Jeziorosy, gdzie zdążono wpisać do akt grodzkich akt powstania. Zaskoczenie Dyneburga nie powiodło się powstańcom, młodzi zrewoltowani podchorążowie zostali wysłani do obozu Dybicza. Wobec pogarszającego się położenia militarnego, Emilia zrezygnowała ostatecznie z ataku na Dyneburg. Emilia dzieliła z podkomendnymi wszystk

In [11]:
result

,title,context,question,answers,generated_answer
id,,,,,
1,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,Co było powodem powrócenia konceptu porozumien...,"{'text': ['wymianą listów Ripka – Stroński'], ...",規.̋: Konfederacja polsko-czechosłowacka誰: Konf...
2,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,Pomiędzy jakimi stronami odbyło się zgromadzen...,{'text': ['E. Beneša i J. Masaryka z jednej a ...,"規 w sierpniu 1942 roku? odpowiedź: Tak, w sier..."
3,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,O co ubiegali się polscy przedstawiciele podcz...,{'text': ['podpisanie układu konfederacyjnego'...,規: Konfederacja polsko-czechosłowacka 1942̋: K...
4,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,Który z dyplomatów sprzeciwił się konceptowi k...,"{'text': ['E. Beneš'], 'answer_start': [558]}",規: Konfederacja polsko-czechosłowacka w listop...
5,Konfederacja polsko-czechosłowacka,Projekty konfederacji zaczęły się załamywać 5 ...,Kiedy oficjalnie doszło do zawarcia porozumienia?,"{'text': ['20 listopada 1942'], 'answer_start'...",規: 20 listopada 1942 r.̋: 20 listopada 1942 r....
6,Pomilio PD/PE,Samoloty Pomilio PD weszły na wyposażenie jedn...,Jaki model zastąpił Pomilio PC?,"{'text': ['Pomilio PD'], 'answer_start': [9]}",規: Pomilio PD/PE̋: Pomilio PC
7,Pomilio PD/PE,Samoloty Pomilio PD weszły na wyposażenie jedn...,Co wchodziło w skład eskadry 139?,{'text': ['dwóch sekcji początkowo przydzielon...,規.̋: Pomilio PD/PE誰: Pomilio PD/PE.
8,Pomilio PD/PE,Samoloty Pomilio PD weszły na wyposażenie jedn...,Jaki model zastąpił samoloty Pomilio PD?,"{'text': ['PE'], 'answer_start': [683]}",規: Pomilio PD/PE?
9,Pomilio PD/PE,"Na początku 1918 roku planowano, że Pomilio PD...",Czym według założeń z 1918 roku miały zajmować...,{'text': ['wyposażeniem tylko armijnych eskadr...,規: Pomilio PD/PE̋: Pomilio PD/PE


In [36]:
import json

In [35]:
pd.read_json("../data/poquad-downloaded-manually/poquad-dev.json").iloc[0].data

{'id': 9773,
 'title': 'Miszna',
 'summary': 'Miszna (hebr.\xa0\u200fמשנה\u200e miszna „nauczać”, „ustnie przekazywać”, „studiować”, „badać”, od \u200fשנה\u200e szana „powtarzać”, „różnić się”, „być odmiennym”; jid. Miszne) – w judaizmie uporządkowany zbiór tekstów ustnego prawa uzupełniający Torę (Prawo pisane). Według wierzeń judaizmu stanowi ustną, niespisaną część prawa nadanego przez Boga na Synaju, tzw. Torę ustną. Jest świętym tekstem judaizmu i jest traktowana na równi z Tanach (Biblią hebrajską). Zbiór był w Izraelu od wieków przekazywany ustnie z pokolenia na pokolenie, zwiększył swój rozmiar szczególnie w okresie od III w. p.n.e. do II w. n.e. w wyniku systematycznego uzupełniania komentarzy przez tannaitów, żydowskich nauczycieli prawa ustnego. Miszna została spisana dopiero w II–III w. Prace redakcyjne zapoczątkował rabin Akiba ben Josef, a kształt ostatecznej redakcji tekstu nadał Juda ha-Nasi. Miszna składa się z 6 porządków (hebr.: sedarim), które dzielą się na 63 trakt

Script Loading Poquad data on huggingface is broken. So manually downloading data and processing it differently is required.